In [ ]:
from predict import perform_inference
from datasets.rfw_latent import RFW_raw, RFW_latent, create_dataloaders
from train import train, write_model, save_model
from detection_models import MultiHeadResNet, LatentMultiHead_1

import torch
import torchvision.models as models
import torch.nn as nn
from tqdm import tqdm
import torch.nn.functional as F

import sys
sys.path.append('/home/tianqiu/NeuralCompression/lossy-vae')
from lvae import get_model
from lvae.models.qresvae import zoo

In [2]:
DEVICE = 1
device = torch.device(f'cuda:{DEVICE}' if torch.cuda.is_available() else 'cpu')
RATIO = 0.8

In [3]:
BATCH_SIZE = 1024

In [4]:
# load pre-trained qres model
model_name = 'qres17m'
lmb_value = 64
nc_model = get_model(model_name, lmb_value, pretrained=True).to(device) # weights are downloaded automatically

In [5]:
RFW_IMAGES_DIR =  "/media/global_data/fair_neural_compression_data/datasets/RFW/data_64"
RFW_LABELS_DIR = "/media/global_data/fair_neural_compression_data/datasets/RFW/clean_metadata/numerical_labels.csv"
image_ds = RFW_raw(RFW_IMAGES_DIR, RFW_LABELS_DIR)
image_dl_train, image_dl_test = create_dataloaders(image_ds, BATCH_SIZE, RATIO)
latent_ds = RFW_latent(RFW_IMAGES_DIR, RFW_LABELS_DIR, nc_model, device)
latent_dl_train, latent_dl_test = create_dataloaders(latent_ds, BATCH_SIZE, RATIO)

In [6]:
# keep latents with n_keep
def get_latent(img, nc_model, device, n_keep=12):
        ps_layer = nn.PixelShuffle(2)
        img = img.to(device)
        # print(img.shape)
        stats_all = nc_model.forward_get_latents(img)
        latents = [stats_all[latent_block_index]['z'] for latent_block_index in range(12)]
        if n_keep == 12:
                output = torch.cat((F.interpolate(latents[0], 4),latents[1], latents[2]), 1)
                output = torch.cat((F.interpolate(output, 8),latents[3], latents[4],latents[5], latents[6]), 1)
                output = torch.cat((F.interpolate(output, 16),latents[7], latents[8],latents[9], latents[10], latents[11]), 1)
                output = ps_layer(output)
        elif n_keep == 1:
                output = latents[0]
        return output

In [7]:
output_dims = {
    'skin_type': 6,
    'eye_type': 2,
    'nose_type': 2,
    'lip_type': 2,
    'hair_type': 4,
    'hair_color': 5
}

# model = MultiHeadResNet(output_dims).to(device)
model = LatentMultiHead_1(output_dims).to(device)

In [8]:
def train_numerical_rfw(num_epochs, lr, train_loader, device):
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=lr)
    for epoch in range(num_epochs):
        running_loss = 0.0
        with tqdm(total=len(train_loader), desc=f"Epoch {epoch+1}/{num_epochs}") as pbar:
            for inputs, targets, races in train_loader:
                # print(inputs.shape)
                latents = get_latent(inputs, nc_model, device, n_keep=1)
                # inputs, targets = inputs.to(device), targets.to(device)
                targets = targets.to(device)
                optimizer.zero_grad()
                outputs = model(latents)
                loss = 0
                for i, head in enumerate(outputs):
                    loss += criterion(outputs[head], targets[:, i].to(torch.int64))
                loss.backward()
                optimizer.step()
                running_loss += loss.item() * inputs.size(0)
                avg_loss = running_loss / ((pbar.n + 1) * len(latents))  # Compute average loss
                pbar.set_postfix(loss=avg_loss)
                pbar.update(1)
    return model

In [10]:
# model = train_numerical_rfw(10, LEARNING_RATE, latent_dl_train, device)
LEARNING_RATE = 0.01
model = train_numerical_rfw(2000, LEARNING_RATE, image_dl_train, device)
save_model(model, '../models', 'latent_RFW_numerical_all_labels_resnet18')

Epoch 33/2000:  69%|██████▉   | 22/32 [00:12<00:05,  1.83it/s, loss=5.89]


KeyboardInterrupt: 

: 

In [ ]:
import pandas as pd
rfw_labels = pd.read_csv('/media/global_data/fair_neural_compression_data/datasets/RFW/clean_metadata/clean.csv')
rfw_labels.head()

In [ ]:
def select_random_images(df, num_images_per_type=6):
    selected_images = []
    for skintype_col in ['skintype_type1', 'skintype_type2', 'skintype_type3', 'skintype_type4', 'skintype_type5', 'skintype_type6']:
        skin_type = skintype_col.split('_')[-1]  # Extract the skin type from the column name
        # Select images for each skin type
        images = df[df[skintype_col] == 1].sample(num_images_per_type)
        selected_images.extend([[skin_type, class_id, image_path] for class_id, image_path in images[['Class_ID', 'File']].values.tolist()])
    return selected_images

In [ ]:
import matplotlib.pyplot as plt
random_images = select_random_images(rfw_labels)
print(f'random_images: {random_images[:4]}')
num_rows = 6
num_cols = 6
fig, axs = plt.subplots(num_rows, num_cols, figsize=(20, 20))

for i, (skin_type, class_id, image_path) in enumerate(random_images):
    row = i // num_cols
    col = i % num_cols
    img = plt.imread(f'{RFW_IMAGES_DIR}/{class_id}/{image_path}')
    axs[row, col].imshow(img)
    axs[row, col].set_title(f"Skin Type: {skin_type}\nClass ID: {class_id}")
    axs[row, col].axis('off')

plt.tight_layout()
plt.show()

In [ ]:
import torch
# model_path = 'models/RFW_numerical_no_skin_resnet18_2024-05-06_16-07-26.pth'
model_path = '../models/latent_RFW_numerical_all_labels_resnet18_2024-05-08_23-38-13.pth'
model = torch.load(model_path)

In [ ]:
model_name = 'latent_RFW_numerical_all_labels_resnet18_2024-05-08_23-38-13'

In [ ]:
from tqdm import tqdm
import numpy as np

def save_race_based_predictions(
        model, 
        model_name, 
        dataloader, 
        device, 
        prediction_save_dir,
        save_labels=False
    ):
    all_predictions = {'Indian': {head: torch.tensor([]) for head in model.heads.keys()}, 
                       'Caucasian': {head: torch.tensor([]) for head in model.heads.keys()}, 
                       'Asian': {head: torch.tensor([]) for head in model.heads.keys()},  
                       'African': {head: torch.tensor([]) for head in model.heads.keys()}}
    all_labels = {'Indian': {head: torch.tensor([]) for head in model.heads.keys()}, 
                  'Caucasian': {head: torch.tensor([]) for head in model.heads.keys()}, 
                  'Asian': {head: torch.tensor([]) for head in model.heads.keys()}, 
                  'African': {head: torch.tensor([]) for head in model.heads.keys()}}
    
    print(f'prediction_save_dir: {prediction_save_dir}')
    dataloader = tqdm(dataloader, desc="Getting Predictions", unit="batch")
    model.eval()
    with torch.no_grad():
        for _, data in enumerate(dataloader):
            inputs, labels, race = data
            # print(inputs.shape)
            race = np.array(race)
            latents = get_latent(inputs, nc_model, device)
            # inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(latents)

            for i, (head, predictions) in enumerate(outputs.items()):
                head_preds = predictions.argmax(dim=1).cpu()

                for race_label in all_labels:
                    race_indices = np.array((race == race_label).nonzero()[0])
                    race_predictions = head_preds[race_indices]
                    race_labels = labels[:, i][race_indices]
                
                    all_predictions[race_label][head] = torch.cat((all_predictions[race_label][head], race_predictions.to('cpu')), dim=0)
                    all_labels[race_label][head] = torch.cat((all_labels[race_label][head], race_labels.to('cpu')), dim=0)

    for race_label in all_labels:
        for category in all_labels[race_label]:
            torch.save(all_predictions[race_label][category], f'{prediction_save_dir}/{model_name}_{race_label}_{category}_predictions.pt')
            if save_labels:
                torch.save(all_labels[race_label][category], f'{prediction_save_dir}/{model_name}_{race_label}_{category}_labels.pt')

    return all_predictions, all_labels


In [ ]:
all_predictions, all_labels = save_race_based_predictions(model,
                                                model_name,
                                                image_dl_test, 
                                                device, 
                                                '../results/latent_RFW/predictions',
                                            )

# get baseline results

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

all_baseline_results = {}
for race in all_predictions:
    all_baseline_results[race] = {}
    for category in all_predictions[race]:
        all_baseline_results[race][category] = {}
        
        preds = all_predictions[race][category]
        labels = all_labels[race][category]
        all_baseline_results[race][category]['accuracy'] = accuracy_score(labels, preds)
        all_baseline_results[race][category]['precision'] = precision_score(labels, preds, average='weighted')
        all_baseline_results[race][category]['recall'] = recall_score(labels, preds, average='weighted')
        all_baseline_results[race][category]['f1'] = f1_score(labels, preds, average='weighted')

In [ ]:
all_baseline_results

In [ ]:
import matplotlib.pyplot as plt

for race in all_baseline_results:
    
    fig, axs = plt.subplots(1, len(all_baseline_results['Indian']), figsize=(24, 5))
    for i, (category, metrics) in enumerate(all_baseline_results[race].items()):
        axs[i].bar(metrics.keys(), metrics.values())
        axs[i].set_title(category)
        axs[i].set_ylabel('Score')
        axs[i].set_ylim(0, 1)  # Limit y-axis to range [0, 1] for better visualization

    # plt.savefig(f'results/RFW/plots/baselines/baseline_{race}_results.png')
    plt.suptitle(race)
    plt.tight_layout()
    plt.show()